In [1]:
import docx
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from pymorphy2 import MorphAnalyzer
from tqdm.autonotebook import tqdm
import re

tqdm.pandas()

In [2]:
doc_path = 'ESG - абзацы по топикам.docx'
doc = docx.Document(doc_path)

In [3]:
topic = None
extracted_text = []

for paragraph in doc.paragraphs:
    if paragraph.style.name == 'Heading 2':
        topic = paragraph.text
        continue
    elif paragraph.style.name == 'Heading 1':
        continue
    
    if topic is not None:
        extracted_text.append({"topic": topic, "text": paragraph.text})

In [4]:
df = pd.DataFrame(extracted_text)
df.head()

,topic,text
0,Климат,Масса выбросов ПГ сократилась за счет реализац...
1,Климат,В 2022 г. была разработана Климатическая дорож...
2,Климат,В 2022 году Эн+ продолжила работу по борьбе с ...
3,Климат,Хотя периметр потенциального разделения низко ...
4,Климат,- активы с относительно низким углеродным след...


In [5]:
RE_DERIVED = re.compile("\w+( -|- |-|! - )\w+")
RE_RUSSIAN_TEXT = re.compile("[а-яА-Я]+")
EXCLUDE_PATTERNS = re.compile("[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+")

stopwords_ru = stopwords.words("russian")

morph = MorphAnalyzer()
cp_names = {
    "ак",
    "акб",
    "акига",
    "алмазэргиэнбанк",
    "алрос",
    "альфабанк",
    "алюминиевый",
    "амвэя",
    "ангарский",
    "анк",
    "ао",
    "архангельский",
    "ас",
    "асэ",
    "атомпроект",
    "атомредметзолото",
    "атомэнергомаш",
    "атомэнергопроект",
    "атомэнергопром",
    "афк",
    "ашан",
    "аэрофлот",
    "балтика",
    "барс",
    "бат",
    "башнефть",
    "билайн",
    "бифид",
    "бф",
    "виктория",
    "владивостокский",
    "внипиэт",
    "волга",
    "волгателек",
    "волжский",
    "восток",
    "втб",
    "выксунский",
    "вымпелком",
    "вэб",
    "газпром",
    "газпромбанк",
    "гарантинвест",
    "генерировать",
    "геннадий",
    "гидропресс",
    "гк",
    "гмк",
    "гнц",
    "головной",
    "гольцблат",
    "грэс",
    "гтлк",
    "губкин",
    "дания",
    "дивизион",
    "евразийский",
    "еврохим",
    "елена",
    "енисейский",
    "еэс",
    "желдортранс",
    "зао",
    "зарубежнефть",
    "ик",
    "илим",
    "имииафрикантовый",
    "инжиниринговый",
    "инккапитал",
    "интер",
    "интернэшнл",
    "информация",
    "иркутск",
    "казаньоргсинтез",
    "казмунайгаз",
    "калининградский",
    "камаз",
    "касперский",
    "кб",
    "кемикалс",
    "ковровский",
    "комстаротс",
    "красцветмет",
    "крафтфудс",
    "кубань",
    "кузбассэнерго",
    "ленэнерго",
    "леруа",
    "лпк",
    "лср",
    "лукойл",
    "мвидео",
    "мдмбанк",
    "мегафон",
    "мерлен",
    "металлоинвест",
    "металлургический",
    "метафракс",
    "ммк",
    "монди",
    "мон’дэлиса",
    "моррис",
    "мосводоканал",
    "московский",
    "мрск",
    "мрф",
    "мсп",
    "мтс",
    "мхк",
    "неманский",
    "нестля",
    "ниаэп",
    "нижнекамскнефтехим",
    "нииар",
    "нипигаз",
    "ниу",
    "нк",
    "нлмк",
    "новатэк",
    "новогор",
    "новосибирский",
    "нордиск",
    "норильский",
    "оао",
    "объединение",
    "огк",
    "окб",
    "окбм",
    "около",
    "омк",
    "они",
    "ооо",
    "пао",
    "пейп",
    "пепеляев",
    "петропавловск",
    "пивоваренный",
    "пивоваров",
    "пигмент",
    "пик",
    "по",
    "полиметалл",
    "приволжье",
    "прикамье",
    "рао",
    "распадский",
    "ргу",
    "реновый",
    "ржд",
    "рксхолдинг",
    "рольф",
    "росатом",
    "росводоканал",
    "роснефть",
    "россеть",
    "российский",
    "россия",
    "ростелеком",
    "ростёха",
    "росэнергоатом",
    "рудна",
    "русало",
    "русгидро",
    "руссинвест",
    "руссия",
    "русый",
    "русь",
    "рф",
    "рязанский",
    "сабмиллер",
    "санофироссия",
    "сахалин",
    "сбербанк",
    "свез",
    "севергазпром",
    "севернефтегазпром",
    "северсталь",
    "сзлк",
    "сибирь",
    "сибурхолдинг",
    "система",
    "ситибанк",
    "ситроникс",
    "совкомбанк",
    "спбаэп",
    "суал",
    "сургутнефтегаз",
    "суэк",
    "схк",
    "сыктывкарский",
    "такед",
    "таманьнефтегаз",
    "татнефть",
    "твэл",
    "тгк",
    "техснабэкспорт",
    "тимченко",
    "тмк",
    "тнк",
    "тнквр",
    "тоаз",
    "трансаэро",
    "трансгаз",
    "транснефть",
    "тюмень",
    "урал",
    "уралкалий",
    "уралсиб",
    "уральский",
    "урсабанк",
    "ухта",
    "фгк",
    "ферреро",
    "фиабанк",
    "филиал",
    "филип",
    "фк",
    "фортум",
    "фосагро",
    "фпк",
    "фск",
    "химконцентрат",
    "центринвест",
    "черкизовый",
    "чусовской",
    "шелл",
    "шереметьево",
    "щуровский",
    "эксонмобила",
    "энела",
    "энергохолдинг",
    "энерджи",
    "эсэфай",
    "южнороссийский",
    "юкос",
    "юникредитбанк",
    "юнипро",
    "яндекс",
    "ятэк",
}


In [6]:
def transform_text(row):
    paragraph = row["text"]
    report_page = RE_DERIVED.sub("", paragraph)
    report_page_lst = word_tokenize(report_page)
    tokens = []
    for word_ in filter(RE_RUSSIAN_TEXT.match, report_page_lst):
        if word_ and word_ not in stopwords_ru and word_ not in cp_names:
            word_ = word_.strip()
            word_ = morph.parse(word_)[0]
            normal_form = word_.normal_form
            tokens.append(normal_form)
    return " ".join(tokens)

In [7]:
df["parsed_text"] = df.progress_apply(transform_text, axis=1)
df.head()

100%|██████████| 824/824 [00:06<00:00, 124.88it/s]


,topic,text,parsed_text
0,Климат,Масса выбросов ПГ сократилась за счет реализац...,масса выброс пг сократиться счёт реализация ме...
1,Климат,В 2022 г. была разработана Климатическая дорож...,в г. разработать климатический дорожный карта ...
2,Климат,В 2022 году Эн+ продолжила работу по борьбе с ...,в год эн+ продолжить работа борьба изменение к...
3,Климат,Хотя периметр потенциального разделения низко ...,хотя периметр потенциальный разделение низко о...
4,Климат,- активы с относительно низким углеродным след...,актив относительно низкий углеродный следом да...


In [10]:
df.to_csv("paragraph_per_topic.csv")